# Бинарная классификация отзывов к фильмам

Подключим библиотеки 

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

Загрузим данные для тренировки и теста 

In [7]:
import os
train_dir=os.getcwd()+"/aclImdb/train"
test_dir=os.getcwd()+"/aclImdb/test"

'/home/dima/imdb_analysis/aclImdb/train'

In [3]:
reviews_train = load_files(y)
text_train, y_train = reviews_train.data, reviews_train.target

In [4]:
print("Number of documents in training data: %d" % len(text_train))
print(np.bincount(y_train))

Number of documents in training data: 25000
[12500 12500]


In [5]:
reviews_test = load_files(x+"aclImdb/test")
text_test, y_test = reviews_test.data, reviews_test.target

FileNotFoundError: [Errno 2] No such file or directory: '/home/dima/imdb_analysisaclImdb/test'

In [ ]:
print("Number of documents in test data: %d" % len(text_test))
print(np.bincount(y_test))

Закодируем данные для дальнейшей работы с ними. Для этого используем vtnjl "Bag of words" и кодификатор CountVectorizer

In [ ]:
cv = CountVectorizer()
cv.fit(text_train)
print(len(cv.vocabulary_))

In [ ]:
print(cv.get_feature_names()[:50])
print(cv.get_feature_names()[50000:50050])

In [ ]:
X_train = cv.transform(text_train)
X_test = cv.transform(text_test)

Обучим логистическую регрессию.

In [ ]:
%%time
logit = LogisticRegression(n_jobs=-1, random_state=7,solver='lbfgs', max_iter=1000)
logit.fit(X_train, y_train)
print(round(logit.score(X_train, y_train), 3), round(logit.score(X_test, y_test), 3))

На тестовой выборке мы правильно определяем тональность примерно 86,7% отзывов

Отобразим коэффициенты модели, т.е. посмотрим, какие слова играли наибольший вес при определении тональности отзыва.

In [ ]:
def visualize_coefficients(classifier, feature_names, n_top_features=25):
# get coefficients with large absolute values 
    coef = classifier.coef_.ravel()
    positive_coefficients = np.argsort(coef)[-n_top_features:]
    negative_coefficients = np.argsort(coef)[:n_top_features]
    interesting_coefficients = np.hstack([negative_coefficients, positive_coefficients])
# plot them
    plt.figure(figsize=(15, 5))
    colors = ["red" if c < 0 else "blue" for c in coef[interesting_coefficients]]
    plt.bar(np.arange(2 * n_top_features), coef[interesting_coefficients], color=colors)
    feature_names = np.array(feature_names)
    plt.xticks(np.arange(1, 1 + 2 * n_top_features), feature_names[interesting_coefficients], rotation=60, ha="right");

In [ ]:
def plot_grid_scores(grid, param_name):
    plt.plot(grid.param_grid[param_name], grid.cv_results_['mean_train_score'],
    color='green', label='train')
    plt.plot(grid.param_grid[param_name], grid.cv_results_['mean_test_score'],
    color='red', label='test')
    plt.legend();

In [ ]:
visualize_coefficients(logit, cv.get_feature_names())